In [4]:
import os
from aocd import get_data, submit1, submit2


day, year = 11, 2016
fn = f'input/Day{day:02}.txt'

if not os.path.exists(fn):
    with open(fn, 'w') as f:
        f.write(get_data(day=day, year=year))
    
with open(fn) as f:
    inp = f.read()

AoC 2016 Day 11
========
https://adventofcode.com/2016/day/11

In [3]:
import numpy as np
import pandas as pd
from itertools import product, combinations, chain

def build(node):
    kind = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1])
    mat = np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4])

    s = np.zeros((4, 2, 5), bool)
    s[node, kind, mat] = True
    return s

def valid(node):
    s = build(node)
    return ~(
        s[:, 0, :] * (1 - s[:, 1, :])
        * s[:, 1].any(1, keepdims=True)
    ).any()

In [120]:
a = np.array(list(product(*(range(4) for _ in range(10)))))
r = np.apply_along_axis(valid, 1, a)

nodes = set(map(tuple, a[r]))

In [225]:
floor = 0
config = np.array([1, 1, 0, 0, 1, 1, 0, 0, 2, 1])

In [368]:
def where_2(floor, config, nodes):
    width = config.shape[0]
    where = np.flatnonzero(config == floor)
    combs = sum(map(list, map(lambda n: combinations(where, n), [1, 2])), [])
    bins  = [i * width + j for i, t in enumerate(combs) for j in t]
    moves = np.bincount(bins, minlength=width * len(combs)).reshape(-1, width)
    
    to = {}
    if floor < 3:
        few = config[None, :] + moves
        few = few[np.array(list(map(lambda a: tuple(a) in nodes, few)))]
        if len(few) > 0:
            to[floor + 1] = few
    if floor > 0:
        few = config[None, :] - moves
        few = few[np.array(list(map(lambda a: tuple(a) in nodes, few)))]
        if len(few) > 0:
            to[floor - 1] = few
    
    return to

In [369]:
config = np.array([1, 1, 0, 0, 1, 1, 0, 0, 2, 1])
goal = (3,) * 10
floor = 0
n = 0
check = (floor,) + tuple(config)
level = {check}
graph = {n: level}
seen = {check}

while goal not in set(map(lambda x: x[1:], level)):
    n += 1
    next_level = set()
    for j, check in enumerate(level):
        floor, *config = check
        config = np.array(config)
        next_level |= {(k,) + tuple(row) for k, v in where_2(floor, config, nodes).items() for row in v}
    next_level -= seen
    seen |= next_level
    level = next_level
    graph[n] = level
    if n % 1 == 0:
        print(f'\r{len(seen)}: {n}: {len(level)}', end='')

158191: 37: 31097

In [8]:
config = np.array([1, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0, 0, 0, 0])
goal = (3,) * 14
floor = 0
where_2(floor, config)

{1: array([[1, 1, 1, 1, 1, 1, 0, 0, 2, 1, 0, 0, 0, 0],
        [1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 0, 0, 0, 0],
        [1, 1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 1, 0, 0],
        [1, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0, 0, 1, 1]])}

In [7]:
def build(node):
    kind = np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1])
    mat = np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6])

    s = np.zeros((4, 2, 7), bool)
    s[node, kind, mat] = True
    return s

def valid(node):
    s = build(node)
    return ~(
        s[:, 0, :] * (1 - s[:, 1, :])
        * s[:, 1].any(1, keepdims=True)
    ).any()

def where_2(floor, config):
    width = config.shape[0]
    where = np.flatnonzero(config == floor)
    combs = sum(map(list, map(lambda n: combinations(where, n), [1, 2])), [])
    bins  = [i * width + j for i, t in enumerate(combs) for j in t]
    moves = np.bincount(bins, minlength=width * len(combs)).reshape(-1, width)
    
    to = {}
    if floor < 3:
        few = config[None, :] + moves
        few = few[list(map(valid, few))]
        if len(few) > 0:
            to[floor + 1] = few
    if floor > 0:
        few = config[None, :] - moves
        few = few[list(map(valid, few))]
        if len(few) > 0:
            to[floor - 1] = few
    
    return to

In [9]:
config = np.array([1, 1, 0, 0, 1, 1, 0, 0, 2, 1, 0, 0, 0, 0])
goal = (3,) * 14
floor = 0
n = 0
check = (floor,) + tuple(config)
level = {check}
graph = {n: level}
seen = {check}

while goal not in set(map(lambda x: x[1:], level)):
    n += 1
    next_level = set()
    for j, check in enumerate(level):
        floor, *config = check
        config = np.array(config)
        next_level |= {(k,) + tuple(row) for k, v in where_2(floor, config).items() for row in v}
    next_level -= seen
    seen |= next_level
    level = next_level
    graph[n] = level
    if n % 1 == 0:
        print(f'\r{len(seen)}: {n}: {len(level)}', end='')

6042507: 61: 573252